In [ ]:
#

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from numba import njit
from sklearn.decomposition import PCA

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

TRAIN=pd.read_csv("../input/jane-street-market-prediction/train.csv")

#test_df=pd.read_csv("../input/jane-street-market-prediction/example_test.csv")

TRAIN=TRAIN.fillna(TRAIN.median())

#TRAIN=TRAIN[TRAIN.feature_0==(-1)]
RESP=TRAIN.resp

TRAIN=TRAIN.drop(["resp_1","resp_2","resp_3","resp_4","date","resp","ts_id","feature_0"],axis=1)

TRAIN=(TRAIN-TRAIN.mean())/TRAIN.std()

MEDIAN=TRAIN.median().to_numpy()

pca = PCA(n_components=110)
TRAIN=pd.DataFrame(pca.fit_transform(TRAIN))

MEDIANPCA=TRAIN.median().to_numpy()

def fillna_npwhere_njit(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

def crd_predict (X_test,PRED,PREDy):
    
    #X_test=X_test.to_numpy()
    ANP=abs(X_test-PRED)
    BNP=((ANP-ANP.min(axis=0))==0)        
    return ((np.sign(np.nansum((PREDy*BNP.T)).T+0.1)+1)/2)

def crd_model (X_train,y_train):
    
    X_train["y"]=y_train
    PRED=X_train.groupby(['y']).median()
    
    return PRED

MODEL=crd_model(TRAIN,abs(RESP)/RESP)

PRED=MODEL.index.to_numpy()

MODEL=MODEL.to_numpy()

del TRAIN 

for (test_df, sample_prediction_df) in iter_test:
    test_df=fillna_npwhere_njit(test_df.drop(["date","feature_0"],axis=1).to_numpy(),MEDIAN)
    sample_prediction_df.action=int(crd_predict(pca.transform(test_df),MODEL,PRED))
    env.predict(sample_prediction_df)
